In [1]:
import pandas as pd
import pickle

In [ ]:
with open('gencode.v39.transcripts.fa', mode='r', encoding='utf-8') as f:
    df_raw = pd.DataFrame(f.readlines())

In [ ]:
df_raw[0].head()

In [ ]:
name_index = df_raw.index[df_raw[0].str.contains('>')].tolist()
name_index[:10]

In [ ]:
sequences = pd.Series()

In [ ]:
count = 0
for i, line in enumerate(df_raw[0]):
    if (i==[name_index[count]][0]):
        sequences[line] = ''
        count += 1
    else:
        sequences[df_raw[0][name_index[count-1]]] = sequences[df_raw[0][name_index[count-1]]] + df_raw[0][i].rstrip('\n')
    #試作中に行きすぎないよう
    # if (i==500):
    #     break

In [ ]:
sequences

In [ ]:
sequences.to_pickle('sequences.pkl')

## ここからtarget側に入ります

In [2]:
with open('human_retry.txt', mode='r', encoding='utf-8') as f:
    df_raw = pd.DataFrame(f.readlines())

In [3]:
df_raw.head

<bound method NDFrame.head of                                                            0
0          >ENST00000456328.2|ENSG00000223972.5|OTTHUMG00...
1                                         1652\t5,2.75963;\n
2                                         1651\t5,3.74253;\n
3                                         1650\t5,6.49098;\n
4                                          1649\t5,8.5129;\n
...                                                      ...
411439763                                    3\t5,8.06935;\n
411439764                                    2\t5,9.07034;\n
411439765                                    1\t5,9.08644;\n
411439766                                    0\t5,9.05754;\n
411439767                                                 \n

[411439768 rows x 1 columns]>

In [4]:
name_index = df_raw.index[df_raw[0].str.contains('>')].tolist()
len(name_index)

244939

In [5]:
sequences = pd.Series()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [6]:
import re

In [7]:
count = 0
for i, line in enumerate(df_raw[0]):
    if (i==[name_index[count]][0]):
        sequences[line] = ''
        count += 1
    else:
        sequences[df_raw[0][name_index[count-1]]] = sequences[df_raw[0][name_index[count-1]]] + re.sub(r'\d+\t5','',df_raw[0][i]).rstrip(';\n')
    # if (i==50000):
    #     break

IndexError: list index out of range

In [8]:
sequences

>ENST00000456328.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000362751.1|DDX11L1-202|DDX11L1|1657|processed_transcript|\n                 ,2.75963,3.74253,6.49098,8.5129,8.96709,9.0653...
>ENST00000450305.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000002844.2|DDX11L1-201|DDX11L1|632|transcribed_unprocessed_pseudogene|\n    ,2.258,2.30794,2.33991,2.33876,2.46248,1.23872...
>ENST00000488147.1|ENSG00000227232.5|OTTHUMG00000000958.1|OTTHUMT00000002839.1|WASH7P-201|WASH7P|1351|unprocessed_pseudogene|\n                 ,2.22498,2.35681,2.55085,2.58041,2.62294,2.605...
>ENST00000619216.1|ENSG00000278267.1|-|-|MIR6859-1-201|MIR6859-1|68|miRNA|\n                                                                    ,2.00031,5.383,6.19813,7.72055,7.97902,7.97111...
>ENST00000473358.1|ENSG00000243485.5|OTTHUMG00000000959.2|OTTHUMT00000002840.1|MIR1302-2HG-202|MIR1302-2HG|712|lncRNA|\n                        ,0.056234,0.0544085,0.054602,0.0486438,0.05109...
                              

In [9]:
sequences.to_pickle('human_target.pkl')

In [11]:
!ls -s -lh

total 17G
427M -rw-r--r-- 1 kaisei-h mh-ut 427M Dec 15 08:02 gencode.v39.transcripts.fa
252M -rw-r--r-- 1 kaisei-h mh-ut 252M Dec 15 08:01 gencode.vM28.transcripts.fa
4.0K -rw-r--r-- 1 kaisei-h mh-ut  359 Dec 21 09:37 human.sh
4.0K -rw-r--r-- 1 kaisei-h mh-ut   51 Dec 30 09:27 human.sh.e14374651
4.0K -rw-r--r-- 1 kaisei-h mh-ut   98 Dec 30 09:27 human.sh.o14374651
6.0G -rw-r--r-- 1 kaisei-h mh-ut 6.0G Dec 30 09:27 human_retry.txt
424M -rw-r--r-- 1 kaisei-h mh-ut 424M Jan  7 06:59 human_seq.pkl
3.2G -rw-r--r-- 1 kaisei-h mh-ut 3.2G Jan  7 06:48 human_target.pkl
4.0K -rw-r--r-- 1 kaisei-h mh-ut  360 Dec 21 09:37 mouse.sh
4.0K -rw-r--r-- 1 kaisei-h mh-ut   51 Dec 26 18:09 mouse.sh.e14374652
4.0K -rw-r--r-- 1 kaisei-h mh-ut   98 Dec 26 18:09 mouse.sh.o14374652
3.6G -rw-r--r-- 1 kaisei-h mh-ut 3.6G Dec 26 18:09 mouse_retry.txt
250M -rw-r--r-- 1 kaisei-h mh-ut 250M Jan  7 06:56 mouse_seq.pkl
1.9G -rw-r--r-- 1 kaisei-h mh-ut 1.9G Jan  6 16:00 mouse_target.pkl
424M -rw-r--r-- 1 kaisei-h mh-ut 

## ここからはマージします

In [37]:
human_target = pickle.load(open("human_target.pkl","rb"))
human_seq = pickle.load(open("human_seq.pkl","rb"))

In [38]:
human_target.head

<bound method NDFrame.head of >ENST00000456328.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000362751.1|DDX11L1-202|DDX11L1|1657|processed_transcript|\n                 ,2.75963,3.74253,6.49098,8.5129,8.96709,9.0653...
>ENST00000450305.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000002844.2|DDX11L1-201|DDX11L1|632|transcribed_unprocessed_pseudogene|\n    ,2.258,2.30794,2.33991,2.33876,2.46248,1.23872...
>ENST00000488147.1|ENSG00000227232.5|OTTHUMG00000000958.1|OTTHUMT00000002839.1|WASH7P-201|WASH7P|1351|unprocessed_pseudogene|\n                 ,2.22498,2.35681,2.55085,2.58041,2.62294,2.605...
>ENST00000619216.1|ENSG00000278267.1|-|-|MIR6859-1-201|MIR6859-1|68|miRNA|\n                                                                    ,2.00031,5.383,6.19813,7.72055,7.97902,7.97111...
>ENST00000473358.1|ENSG00000243485.5|OTTHUMG00000000959.2|OTTHUMT00000002840.1|MIR1302-2HG-202|MIR1302-2HG|712|lncRNA|\n                        ,0.056234,0.0544085,0.054602,0.0486438,0.05109...


In [39]:
human_seq.head

<bound method NDFrame.head of >ENST00000456328.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000362751.1|DDX11L1-202|DDX11L1|1657|processed_transcript|\n                 GTTAACTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCAT...
>ENST00000450305.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000002844.2|DDX11L1-201|DDX11L1|632|transcribed_unprocessed_pseudogene|\n    GTGTCTGACTTCCAGCAACTGCTGGCCTGTGCCAGGGTGCAAGCTG...
>ENST00000488147.1|ENSG00000227232.5|OTTHUMG00000000958.1|OTTHUMT00000002839.1|WASH7P-201|WASH7P|1351|unprocessed_pseudogene|\n                 ATGGGAGCCGTGTGCACGTCGGGAGCTCGGAGTGAGCGCACCATGA...
>ENST00000619216.1|ENSG00000278267.1|-|-|MIR6859-1-201|MIR6859-1|68|miRNA|\n                                                                    TGTGGGAGAGGAACATGGGCTCAGGACAGCGGGTGTCAGCTTGCCT...
>ENST00000473358.1|ENSG00000243485.5|OTTHUMG00000000959.2|OTTHUMT00000002840.1|MIR1302-2HG-202|MIR1302-2HG|712|lncRNA|\n                        GTGCACACGGCTCCCATGCGTTGTCTTCCGAGCGTCAGGCCGCCCC...


In [40]:
df = pd.concat([human_seq, human_target], axis=1)

In [41]:
df = df.rename(columns={0:'seq', 1:'target'})

In [42]:
df

,seq,target
>ENST00000456328.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000362751.1|DDX11L1-202|DDX11L1|1657|processed_transcript|\n,GTTAACTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCAT...,",2.75963,3.74253,6.49098,8.5129,8.96709,9.0653..."
>ENST00000450305.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000002844.2|DDX11L1-201|DDX11L1|632|transcribed_unprocessed_pseudogene|\n,GTGTCTGACTTCCAGCAACTGCTGGCCTGTGCCAGGGTGCAAGCTG...,",2.258,2.30794,2.33991,2.33876,2.46248,1.23872..."
>ENST00000488147.1|ENSG00000227232.5|OTTHUMG00000000958.1|OTTHUMT00000002839.1|WASH7P-201|WASH7P|1351|unprocessed_pseudogene|\n,ATGGGAGCCGTGTGCACGTCGGGAGCTCGGAGTGAGCGCACCATGA...,",2.22498,2.35681,2.55085,2.58041,2.62294,2.605..."
>ENST00000619216.1|ENSG00000278267.1|-|-|MIR6859-1-201|MIR6859-1|68|miRNA|\n,TGTGGGAGAGGAACATGGGCTCAGGACAGCGGGTGTCAGCTTGCCT...,",2.00031,5.383,6.19813,7.72055,7.97902,7.97111..."
>ENST00000473358.1|ENSG00000243485.5|OTTHUMG00000000959.2|OTTHUMT00000002840.1|MIR1302-2HG-202|MIR1302-2HG|712|lncRNA|\n,GTGCACACGGCTCCCATGCGTTGTCTTCCGAGCGTCAGGCCGCCCC...,",0.056234,0.0544085,0.054602,0.0486438,0.05109..."
...,...,...
>ENST00000361681.2|ENSG00000198695.2|-|-|MT-ND6-201|MT-ND6|525|protein_coding|\n,ATGATGTATGCTTTGTTTCTGTTGAGTGTGGGTTTAGTAATGGGGT...,",0.49418,0.517246,0.532746,0.587294,0.662901,0..."
>ENST00000387459.1|ENSG00000210194.1|-|-|MT-TE-201|MT-TE|69|Mt_tRNA|\n,GTTCTTGTAGTTGAAATACAACGATGGTTTTTCATATCATTGGTCG...,",6.47313,7.3187,7.65946,8.78385,8.80194,8.1545..."
>ENST00000361789.2|ENSG00000198727.2|-|-|MT-CYB-201|MT-CYB|1141|protein_coding|\n,ATGACCCCAATACGCAAAACTAACCCCCTAATAAAATTAATTAACC...,",4.19049,4.35228,4.37056,4.37191,4.55269,4.229..."
>ENST00000387460.2|ENSG00000210195.2|-|-|MT-TT-201|MT-TT|66|Mt_tRNA|\n,GTCCTTGTAGTATAAACTAATACACCAGTCTTGTAAACCGGAGATG...,",8.53848,8.54773,8.5083,9.56009,8.49207,4.7275..."


In [43]:
df.to_pickle('human.pkl')

In [44]:
!ls -s -lh

total 22G
427M -rw-r--r-- 1 kaisei-h mh-ut 427M Dec 15 08:02 gencode.v39.transcripts.fa
252M -rw-r--r-- 1 kaisei-h mh-ut 252M Dec 15 08:01 gencode.vM28.transcripts.fa
3.6G -rw-r--r-- 1 kaisei-h mh-ut 3.6G Jan  7 07:39 human.pkl
4.0K -rw-r--r-- 1 kaisei-h mh-ut  359 Dec 21 09:37 human.sh
4.0K -rw-r--r-- 1 kaisei-h mh-ut   51 Dec 30 09:27 human.sh.e14374651
4.0K -rw-r--r-- 1 kaisei-h mh-ut   98 Dec 30 09:27 human.sh.o14374651
6.0G -rw-r--r-- 1 kaisei-h mh-ut 6.0G Dec 30 09:27 human_retry.txt
424M -rw-r--r-- 1 kaisei-h mh-ut 424M Jan  7 06:59 human_seq.pkl
3.2G -rw-r--r-- 1 kaisei-h mh-ut 3.2G Jan  7 06:48 human_target.pkl
2.1G -rw-r--r-- 1 kaisei-h mh-ut 2.1G Jan  7 07:37 mouse.pkl
4.0K -rw-r--r-- 1 kaisei-h mh-ut  360 Dec 21 09:37 mouse.sh
4.0K -rw-r--r-- 1 kaisei-h mh-ut   51 Dec 26 18:09 mouse.sh.e14374652
4.0K -rw-r--r-- 1 kaisei-h mh-ut   98 Dec 26 18:09 mouse.sh.o14374652
3.6G -rw-r--r-- 1 kaisei-h mh-ut 3.6G Dec 26 18:09 mouse_retry.txt
250M -rw-r--r-- 1 kaisei-h mh-ut 250M Jan  7